## K겹 오차검증

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [2]:
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

In [3]:
df = pd.read_csv('./dataset/sonar.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [4]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]
print(X)
print(Y_obj)

[[0.02 0.0371 0.0428 ... 0.0084 0.009 0.0032]
 [0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 ...
 [0.0522 0.0437 0.018 ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049 ... 0.0079 0.0036 0.0048]
 [0.026 0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 

In [5]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
print(Y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [7]:
# 빈 accuracy 배열
accuracy = []

In [8]:
# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

Epoch 1/100
187/187 [==============================] - 0s 619us/step - loss: 0.2469 - accuracy: 0.5348
Epoch 2/100
187/187 [==============================] - 0s 171us/step - loss: 0.2282 - accuracy: 0.7326
Epoch 3/100
187/187 [==============================] - 0s 180us/step - loss: 0.2174 - accuracy: 0.6738
Epoch 4/100
187/187 [==============================] - 0s 180us/step - loss: 0.1974 - accuracy: 0.7219
Epoch 5/100
187/187 [==============================] - 0s 181us/step - loss: 0.1887 - accuracy: 0.7701
Epoch 6/100
187/187 [==============================] - 0s 176us/step - loss: 0.1745 - accuracy: 0.8075
Epoch 7/100
187/187 [==============================] - 0s 178us/step - loss: 0.1629 - accuracy: 0.8182
Epoch 8/100
187/187 [==============================] - 0s 182us/step - loss: 0.1558 - accuracy: 0.7914
Epoch 9/100
187/187 [==============================] - 0s 176us/step - loss: 0.1471 - accuracy: 0.8128
Epoch 10/100
187/187 [==============================] - 0s 171us/step - l

187/187 [==============================] - 0s 169us/step - loss: 0.0200 - accuracy: 0.9893
Epoch 80/100
187/187 [==============================] - 0s 181us/step - loss: 0.0198 - accuracy: 0.9893
Epoch 81/100
187/187 [==============================] - 0s 171us/step - loss: 0.0188 - accuracy: 0.9733
Epoch 82/100
187/187 [==============================] - 0s 175us/step - loss: 0.0195 - accuracy: 0.9893
Epoch 83/100
187/187 [==============================] - 0s 171us/step - loss: 0.0173 - accuracy: 0.9947
Epoch 84/100
187/187 [==============================] - 0s 176us/step - loss: 0.0177 - accuracy: 0.9893
Epoch 85/100
187/187 [==============================] - 0s 181us/step - loss: 0.0155 - accuracy: 0.9947
Epoch 86/100
187/187 [==============================] - 0s 176us/step - loss: 0.0146 - accuracy: 0.9947
Epoch 87/100
187/187 [==============================] - 0s 176us/step - loss: 0.0147 - accuracy: 0.9947
Epoch 88/100
187/187 [==============================] - 0s 185us/step - loss:

187/187 [==============================] - 0s 150us/step - loss: 0.0680 - accuracy: 0.9251
Epoch 58/100
187/187 [==============================] - 0s 144us/step - loss: 0.0683 - accuracy: 0.9144
Epoch 59/100
187/187 [==============================] - 0s 149us/step - loss: 0.0601 - accuracy: 0.9358
Epoch 60/100
187/187 [==============================] - 0s 159us/step - loss: 0.0598 - accuracy: 0.9198
Epoch 61/100
187/187 [==============================] - 0s 158us/step - loss: 0.0579 - accuracy: 0.9358
Epoch 62/100
187/187 [==============================] - 0s 150us/step - loss: 0.0570 - accuracy: 0.9412
Epoch 63/100
187/187 [==============================] - 0s 166us/step - loss: 0.0545 - accuracy: 0.9412
Epoch 64/100
187/187 [==============================] - 0s 162us/step - loss: 0.0602 - accuracy: 0.9358
Epoch 65/100
187/187 [==============================] - 0s 164us/step - loss: 0.0555 - accuracy: 0.9412
Epoch 66/100
187/187 [==============================] - 0s 158us/step - loss:

187/187 [==============================] - 0s 155us/step - loss: 0.0889 - accuracy: 0.8717
Epoch 36/100
187/187 [==============================] - 0s 149us/step - loss: 0.0860 - accuracy: 0.9198
Epoch 37/100
187/187 [==============================] - 0s 152us/step - loss: 0.0875 - accuracy: 0.9091
Epoch 38/100
187/187 [==============================] - 0s 161us/step - loss: 0.0874 - accuracy: 0.8930
Epoch 39/100
187/187 [==============================] - 0s 157us/step - loss: 0.0820 - accuracy: 0.9198
Epoch 40/100
187/187 [==============================] - 0s 149us/step - loss: 0.0800 - accuracy: 0.9037
Epoch 41/100
187/187 [==============================] - 0s 160us/step - loss: 0.0793 - accuracy: 0.9144
Epoch 42/100
187/187 [==============================] - 0s 159us/step - loss: 0.0735 - accuracy: 0.9251
Epoch 43/100
187/187 [==============================] - 0s 160us/step - loss: 0.0706 - accuracy: 0.9358
Epoch 44/100
187/187 [==============================] - 0s 148us/step - loss:

187/187 [==============================] - 0s 192us/step - loss: 0.1627 - accuracy: 0.7861
Epoch 14/100
187/187 [==============================] - 0s 187us/step - loss: 0.1496 - accuracy: 0.8128
Epoch 15/100
187/187 [==============================] - 0s 192us/step - loss: 0.1511 - accuracy: 0.7968
Epoch 16/100
187/187 [==============================] - 0s 181us/step - loss: 0.1436 - accuracy: 0.8128
Epoch 17/100
187/187 [==============================] - 0s 160us/step - loss: 0.1409 - accuracy: 0.8128
Epoch 18/100
187/187 [==============================] - 0s 187us/step - loss: 0.1347 - accuracy: 0.8396
Epoch 19/100
187/187 [==============================] - 0s 192us/step - loss: 0.1333 - accuracy: 0.8396
Epoch 20/100
187/187 [==============================] - 0s 181us/step - loss: 0.1340 - accuracy: 0.8235
Epoch 21/100
187/187 [==============================] - 0s 171us/step - loss: 0.1290 - accuracy: 0.8449
Epoch 22/100
187/187 [==============================] - 0s 181us/step - loss:

...

187/187 [==============================] - 0s 161us/step - loss: 0.1151 - accuracy: 0.8556
Epoch 26/100
187/187 [==============================] - 0s 170us/step - loss: 0.1120 - accuracy: 0.8342
Epoch 27/100
187/187 [==============================] - 0s 166us/step - loss: 0.1105 - accuracy: 0.8610
Epoch 28/100
187/187 [==============================] - 0s 172us/step - loss: 0.1084 - accuracy: 0.8503
Epoch 29/100
187/187 [==============================] - 0s 159us/step - loss: 0.1044 - accuracy: 0.8930
Epoch 30/100
187/187 [==============================] - 0s 154us/step - loss: 0.1022 - accuracy: 0.8556
Epoch 31/100
187/187 [==============================] - 0s 159us/step - loss: 0.1123 - accuracy: 0.8342
Epoch 32/100
187/187 [==============================] - 0s 160us/step - loss: 0.1034 - accuracy: 0.8449
Epoch 33/100
187/187 [==============================] - 0s 167us/step - loss: 0.0978 - accuracy: 0.8877
Epoch 34/100
187/187 [==============================] - 0s 159us/step - loss:

187/187 [==============================] - 0s 165us/step - loss: 0.2275 - accuracy: 0.7326
Epoch 4/100
187/187 [==============================] - 0s 178us/step - loss: 0.2169 - accuracy: 0.7273
Epoch 5/100
187/187 [==============================] - 0s 163us/step - loss: 0.2124 - accuracy: 0.7219
Epoch 6/100
187/187 [==============================] - 0s 163us/step - loss: 0.2051 - accuracy: 0.7059
Epoch 7/100
187/187 [==============================] - 0s 171us/step - loss: 0.1919 - accuracy: 0.7433
Epoch 8/100
187/187 [==============================] - 0s 163us/step - loss: 0.1851 - accuracy: 0.7594
Epoch 9/100
187/187 [==============================] - 0s 165us/step - loss: 0.1804 - accuracy: 0.7326
Epoch 10/100
187/187 [==============================] - 0s 163us/step - loss: 0.1735 - accuracy: 0.7594
Epoch 11/100
187/187 [==============================] - 0s 160us/step - loss: 0.1652 - accuracy: 0.7914
Epoch 12/100
187/187 [==============================] - 0s 165us/step - loss: 0.161

Epoch 82/100
187/187 [==============================] - 0s 133us/step - loss: 0.0397 - accuracy: 0.9626
Epoch 83/100
187/187 [==============================] - 0s 133us/step - loss: 0.0391 - accuracy: 0.9679
Epoch 84/100
187/187 [==============================] - 0s 149us/step - loss: 0.0370 - accuracy: 0.9679
Epoch 85/100
187/187 [==============================] - 0s 149us/step - loss: 0.0395 - accuracy: 0.9733
Epoch 86/100
187/187 [==============================] - 0s 155us/step - loss: 0.0404 - accuracy: 0.9786
Epoch 87/100
187/187 [==============================] - 0s 139us/step - loss: 0.0363 - accuracy: 0.9679
Epoch 88/100
187/187 [==============================] - 0s 140us/step - loss: 0.0347 - accuracy: 0.9679
Epoch 89/100
187/187 [==============================] - 0s 149us/step - loss: 0.0355 - accuracy: 0.9733
Epoch 90/100
187/187 [==============================] - 0s 156us/step - loss: 0.0329 - accuracy: 0.9786
Epoch 91/100
187/187 [==============================] - 0s 137us

188/188 [==============================] - 0s 148us/step - loss: 0.0632 - accuracy: 0.9309
Epoch 61/100
188/188 [==============================] - 0s 158us/step - loss: 0.0615 - accuracy: 0.9415
Epoch 62/100
188/188 [==============================] - 0s 158us/step - loss: 0.0640 - accuracy: 0.9202
Epoch 63/100
188/188 [==============================] - 0s 147us/step - loss: 0.0607 - accuracy: 0.9468
Epoch 64/100
188/188 [==============================] - 0s 146us/step - loss: 0.0575 - accuracy: 0.9521
Epoch 65/100
188/188 [==============================] - 0s 165us/step - loss: 0.0594 - accuracy: 0.9415
Epoch 66/100
188/188 [==============================] - 0s 158us/step - loss: 0.0583 - accuracy: 0.9415
Epoch 67/100
188/188 [==============================] - 0s 158us/step - loss: 0.0541 - accuracy: 0.9415
Epoch 68/100
188/188 [==============================] - 0s 173us/step - loss: 0.0546 - accuracy: 0.9521
Epoch 69/100
188/188 [==============================] - 0s 169us/step - loss:

188/188 [==============================] - 0s 180us/step - loss: 0.1202 - accuracy: 0.9202
Epoch 39/100
188/188 [==============================] - 0s 154us/step - loss: 0.1168 - accuracy: 0.9043
Epoch 40/100
188/188 [==============================] - 0s 138us/step - loss: 0.1177 - accuracy: 0.9043
Epoch 41/100
188/188 [==============================] - 0s 140us/step - loss: 0.1187 - accuracy: 0.8989
Epoch 42/100
188/188 [==============================] - 0s 146us/step - loss: 0.1124 - accuracy: 0.9096
Epoch 43/100
188/188 [==============================] - 0s 145us/step - loss: 0.1141 - accuracy: 0.9043
Epoch 44/100
188/188 [==============================] - 0s 143us/step - loss: 0.1084 - accuracy: 0.8883
Epoch 45/100
188/188 [==============================] - 0s 142us/step - loss: 0.1077 - accuracy: 0.9202
Epoch 46/100
188/188 [==============================] - 0s 149us/step - loss: 0.1106 - accuracy: 0.9096
Epoch 47/100
188/188 [==============================] - 0s 157us/step - loss: